In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten

from loader import load_20news
from custom_layer import MonteCarloLRF

import numpy as np
import sklearn as sk

Using TensorFlow backend.


In [3]:
(input_shape, nb_classes), (X_train, X_test, Y_train, Y_test), graph_data = \
    load_20news(data_home='data', top_words=1000, sparse=False, remove_short_documents=True, verbose=False)

In [4]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
batch_size = 64
num_classes = Y_train.shape[1]

In [5]:
print(X_train.shape)
print(X_test.shape)
print(batch_size, num_classes)

(9922, 1000, 1)
(6695, 1000, 1)
64 20


In [6]:
METRIC = 'euclidean'
distances = sk.metrics.pairwise.pairwise_distances(graph_data, metric=METRIC, n_jobs=-2)

# enforce exact zero
for k in xrange(distances.shape[0]):
  distances[k,k] = 0.

In [7]:
# max normalize
distances /= distances.max()

# use tricube kernel (becaause of flatness around 0)
probabilities = (1. - np.abs(distances) ** 3) ** 3

# remove auto connections (which are taken anyway in LRF)
for k in xrange(probabilities.shape[0]):
  probabilities[k,k] = 0.

# normalize proba
probabilities /= np.sum(probabilities, axis=1).reshape((probabilities.shape[0], 1))

In [8]:
model = Sequential()
model.add(Conv1D(64, kernel_size=1, activation='relu',
                 padding='same',kernel_initializer='he_uniform',
                 input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(SeparableMonteCarloLRF(probabilities, LRF_size=16, activation='relu'))
model.add(Conv1D(64, kernel_size=1, activation='relu',
                 padding='same',kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(SeparableMonteCarloLRF(probabilities, LRF_size=16, activation='relu'))
model.add(Conv1D(64, kernel_size=1, activation='relu',
                 padding='same',kernel_initializer='he_uniform'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 1000, 64)          128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 64)          0         
_________________________________________________________________
monte_carlo_lrf_1 (MonteCarl (None, 1000, 64)          1088      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1000, 64)          4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 64)          0         
_________________________________________________________________
monte_carlo_lrf_2 (MonteCarl (None, 1000, 64)          1088      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 1000, 64)          4160      
__________

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 9922 samples, validate on 6695 samples
Epoch 1/20
9922/9922 [==============================] - 60s - loss: 2.4232 - acc: 0.2991 - val_loss: 1.8613 - val_acc: 0.4796
Epoch 2/20
9922/9922 [==============================] - 59s - loss: 1.4194 - acc: 0.5972 - val_loss: 1.7020 - val_acc: 0.5144
Epoch 3/20
9922/9922 [==============================] - 59s - loss: 1.0990 - acc: 0.6759 - val_loss: 1.7479 - val_acc: 0.5223
Epoch 4/20
9922/9922 [==============================] - 59s - loss: 0.8993 - acc: 0.7259 - val_loss: 1.7302 - val_acc: 0.5373
Epoch 5/20
9922/9922 [==============================] - 59s - loss: 0.7542 - acc: 0.7649 - val_loss: 1.7987 - val_acc: 0.5525
Epoch 6/20
9922/9922 [==============================] - 59s - loss: 0.6377 - acc: 0.8029 - val_loss: 1.8577 - val_acc: 0.5298
Epoch 7/20
9922/9922 [==============================] - 59s - loss: 0.5341 - acc: 0.8315 - val_loss: 1.9870 - val_acc: 0.5395
Epoch 8/20
9922/9922 [==============================] - 59s - loss: 0.